In [3]:
import os

In [4]:
%pwd

'c:\\Users\\shalm\\source\\repos\\EndToEndMachineLearningProject\\research'

In [5]:
os.chdir("../")

In [6]:
%pwd

'c:\\Users\\shalm\\source\\repos\\EndToEndMachineLearningProject'

In [7]:

from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str

In [8]:

from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [13]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_trainer_config(self) -> ModelTrainerConfig:  # assigning the return type. return type is created above. 
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])  #create root directory folder

        model_trainer_config = ModelTrainerConfig(  #return the variables from configuration one by one
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            alpha = params.alpha,
            l1_ratio = params.l1_ratio,
            target_column = schema.name
            
        )

        return model_trainer_config

In [10]:
import pandas as pd
import os
from mlProject import logger
from sklearn.linear_model import ElasticNet
import joblib

In [14]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config


    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)


        train_x = train_data.drop([self.config.target_column], axis=1)  #dropping target column and just keeping independent columns as this is train data
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]


        lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))  # save the model in the model trainer folder

In [16]:
try:                          # creating pipeline : sequenceally calling the classes or functions
    config = ConfigurationManager()  #initialise configmanager
    model_trainer_config = config.get_model_trainer_config()  #get model related configurations
    model_trainer_config = ModelTrainer(config=model_trainer_config) #initialise modeltrainer object
    model_trainer_config.train()   # call train data
except Exception as e:
    raise e

[2024-03-19 21:10:53,705: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-19 21:10:53,724: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-19 21:10:53,735: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-03-19 21:10:53,738: INFO: common: created directory at: artifacts]
[2024-03-19 21:10:53,742: INFO: common: created directory at: artifacts/model_trainer]
